In [1]:
import pandas as pd
from django_plotly_dash import DjangoDash
from sqlalchemy import create_engine
from dash import dcc, dash_table, html, Dash
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import geopandas
import shapely
import textwrap
import os
from dotenv import load_dotenv
from functools import lru_cache

# Cargar variables de entorno
load_dotenv()
HOST = os.getenv('DB_HOST', 'localhost')
NAME = os.getenv('DB_NAME', 'elecciones23')
PASS = os.getenv('DB_PASS', '4rcg1s2024')
MAPBOX_TOKEN = os.getenv('MAPBOX_TOKEN', 'pk.eyJ1IjoiemFsaXRvYXIiLCJhIjoiYVJFNTlfbyJ9.mPX8qTsRUGOOETl0CtA-Pg')

# Conexión a la base de datos con manejo de errores
try:
    conn = create_engine(f"postgresql+psycopg2://elecciones23:{PASS}@{HOST}:5432/{NAME}")
except Exception as e:
    print(f"Error conectando a la base de datos: {e}")
    raise

# Consultas SQL
sql_base = """
SELECT tipo, nivel, nivel1, fecha, detalle, subsanada, medidas_adoptadas, distrito, ubicacion as geom
FROM novedades_local {where} ORDER BY fecha ASC
"""
sql_generales_base = """
SELECT tipo, 
       nivel AS nivel1,
       CASE
           WHEN nivel::text = '1'::text THEN 'Media'::text
           WHEN nivel::text = '2'::text THEN 'Alta'::text
           WHEN nivel::text = '3'::text THEN 'Crítica'::text
           ELSE NULL::text
       END AS nivel,
       to_char(fecha, 'dd/mm/YYYY HH:MI'::text) AS fecha, detalle, subsanada, medidas_adoptadas, distrito,
       ubicacion as geom
FROM novedades_generales {where} ORDER BY fecha ASC
"""
sql_distritos = "SELECT nam as distrito FROM distritos"
sql_malvinas = """
SELECT nam, geom, -51.74153477642776 as lat, 59.38576589695881 as lon FROM public.malvinas
"""

# Función para consultas dinámicas con caché
@lru_cache(maxsize=32)
def cached_read_postgis(sql, conn_str, params_tuple=()):
    params = dict(params_tuple)
    return geopandas.read_postgis(sql, conn, params=params).to_crs(4326)

def get_data(distrito=None, subsanada=None):
    where_clauses = []
    params = {}
    if distrito and distrito != 'TODOS':
        where_clauses.append("distrito = %(distrito)s")
        params['distrito'] = distrito
    if subsanada:
        where_clauses.append("subsanada = %(subsanada)s")
        params['subsanada'] = subsanada
    where = "WHERE " + " AND ".join(where_clauses) if where_clauses else ""
    sql = sql_base.format(where=where)
    return cached_read_postgis(sql, str(conn), tuple(params.items()))

def get_data_generales(distrito=None, subsanada=None):
    where_clauses = []
    params = {}
    if distrito and distrito != 'TODOS':
        where_clauses.append("distrito = %(distrito)s")
        params['distrito'] = distrito
    if subsanada:
        where_clauses.append("subsanada = %(subsanada)s")
        params['subsanada'] = subsanada
    where = "WHERE " + " AND ".join(where_clauses) if where_clauses else ""
    sql = sql_generales_base.format(where=where)
    return cached_read_postgis(sql, str(conn), tuple(params.items()))

# Cargar datos iniciales
df = get_data()
df_generales = get_data_generales()
distritos_df = pd.read_sql(sql_distritos, conn)
gdf_poligono = geopandas.read_postgis(sql_malvinas, conn, geom_col='geom').to_crs(epsg=4326)
poligono_geojson = gdf_poligono.__geo_interface__

# Agregar coordenadas
df['lon'] = df['geom'].x
df['lat'] = df['geom'].y
df_generales['lon'] = df_generales['geom'].x
df_generales['lat'] = df_generales['geom'].y

def generar_tabla(df1, id):
    if df1.empty:
        return html.Div("No hay datos para mostrar")
    df1 = df1.drop(columns=['geom'], errors='ignore')
    return dash_table.DataTable(
        data=df1.to_dict('records'),
        columns=[{"name": i, "id": i.lower()} for i in ['Fecha', 'Tipo', 'Detalle']],
        id=id,
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        selected_columns=[],
        selected_rows=[],
        style_table={'overflowX': 'scroll'},
        style_cell={
            'fontSize': 11, 'font-family': 'sans-serif', 'textAlign': 'center',
            'whiteSpace': 'normal', 'overflow': 'hidden', 'textOverflow': 'ellipsis'
        },
        page_action="native",
        page_current=0,
        page_size=8,
        style_data={'color': 'black', 'backgroundColor': 'white'},
        style_data_conditional=[
            {"if": {"statePts": "active"}, "backgroundColor": "rgb(102, 102, 102)", "border": "3px solid white", "color": "white"},
            {"if": {"state": "selected"}, "backgroundColor": "rgba(0, 116, 217, 0.3)"}
        ],
        style_header={'backgroundColor': 'rgb(210, 210, 210)', 'color': 'black', 'fontWeight': 'bold'}
    )

def crear_mapa(df, zoom, lat_cen=None, lon_cen=None):
    px.set_mapbox_access_token(MAPBOX_TOKEN)
    punto = shapely.Point(-59.5236, -51.7963)
    gdf_circulo = geopandas.GeoDataFrame(geometry=[punto], crs="EPSG:4326")
    gdf_circulo = gdf_circulo.to_crs("EPSG:32721")
    gdf_circulo['geometry'] = gdf_circulo.geometry.buffer(280000)
    gdf_circulo = gdf_circulo.to_crs("EPSG:4326")
    
    center = dict(lat=lat_cen, lon=lon_cen) if lat_cen and lon_cen else dict(lat=df['lat'].mean() if not df.empty else -34.603722, lon=df['lon'].mean() if not df.empty else -58.381592)
    
    df['detalle2'] = ['<br>'.join(textwrap.wrap(x, width=50)) for x in df['detalle']]
    df['medidas2'] = ['<br>'.join(textwrap.wrap(x, width=50)) for x in df['medidas_adoptadas']]
    
    color_map = {"Media": "#b5c717", "Alta": "#FF9900", "Crítica": "#FF1E00"}
    
    mapa = px.scatter_mapbox(
        data_frame=df, lat="lat", lon="lon",
        hover_name="nivel", color='nivel',
        category_orders={'nivel': ['Media', 'Alta', 'Crítica']},
        color_discrete_map=color_map,
        hover_data=dict(
            distrito=True, tipo=True, fecha=True, detalle2=True,
            subsanada=True, medidas2=True, lat=False, lon=False
        ),
        zoom=zoom, height=800, center=center
    )
    
    mapa.update_traces(
        marker={'size': 13},
        hovertemplate="<br>".join([
            "<b>Nombre:</b> %{customdata[2]}",
            "<b>Distrito:</b> %{customdata[0]}",
            "<b>Tipo:</b> %{customdata[1]}",
            "<b>Fecha:</b> %{customdata[2]}",
            "<b>Detalle:</b> %{customdata[3]}",
            "<b>Subsanada:</b> %{customdata[4]}",
            "<b>Medidas Adoptadas:</b> %{customdata[5]}"
        ])
    )
    
    mapa.update_layout(
        hoverlabel_bgcolor="#FFFFFF", coloraxis_showscale=False,
        mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
    )
    
    mapa.add_choroplethmapbox(
        geojson=gdf_circulo.__geo_interface__,
        locations=gdf_circulo.index,
        z=[1] * len(gdf_circulo),
        colorscale=[[0, 'rgba(170, 211, 223, 1)'], [1, 'rgba(170, 211, 223, 1)']],
        showscale=False, hoverinfo='text', text=['Islas Malvinas'] * len(gdf_poligono),
        marker_line_color='rgba(170, 211, 223, 0.6)', marker_line_width=1
    )
    
    mapa.add_choroplethmapbox(
        geojson=poligono_geojson,
        locations=gdf_poligono.index,
        z=[1] * len(gdf_poligono),
        colorscale=[[0, 'rgb(242, 239, 233, 0.6)'], [1, 'rgb(242, 239, 233, 0.6)']],
        showscale=False, hoverinfo='text', text=['Islas Malvinas'] * len(gdf_poligono)
    )
    
    return mapa

def graf_nov_loc(df):
    estado_counts = df['nivel'].value_counts()
    df_pie = pd.DataFrame({'Nivel': estado_counts.index, 'Cantidad': estado_counts.values})
    color_map = {"Media": "#b5c717", "Alta": "#FF9900", "Crítica": "#FF1E00"}
    
    fig = px.pie(
        df_pie, values='Cantidad', names='Nivel', title='Novedades en locales',
        hole=.4
    )
    
    fig.update_traces(
        textposition='inside', textinfo='percent+label',
        marker=dict(colors=[color_map.get(x, '#000000') for x in df_pie['Nivel']])
    )
    
    fig.update_layout( plot_bgcolor='rgb(40, 40, 40)', title_x=0.5,autosize=True, height=400,
                               paper_bgcolor='rgb(40, 40, 40)', title_font_size=25, font_color='rgb(255, 255, 255)',showlegend=True,font_size=10
                               ,legend=dict(orientation="h", ))
    
    return fig

app2 = Dash(external_stylesheets=[dbc.themes.DARKLY])

app2.layout = html.Div([
    dcc.Store(id='row-count', data={'count': len(df)}),
    dbc.Alert("Hay Novedades Nuevas!!", id="alerta", is_open=False, color="danger", duration=4000),
    html.Audio(id='audio', autoPlay=False, src='/elecciones/static/notificacion.mp3'),
    dbc.Row([html.H2('NOVEDADES', style={'textAlign': 'center', 'background-color': '#267300'})], className="g-0"),
    dbc.Row([
        dbc.Col([
            dbc.Select(
                id="selec_distrito",
                options=[{"label": 'TODOS', "value": 'TODOS'}] +
                        [{"label": x, "value": x} for x in distritos_df['distrito'].drop_duplicates().sort_values()],
                placeholder='DISTRITO'
            )
        ]),
        dbc.Col([
            dbc.Select(
                id="Subsanada",
                options=[{"label": 'No', "value": 'No'}, {"label": 'Sí', "value": 'Sí'}],
                placeholder='SUBSANADO'
            )
        ])
    ], className="m-2 g-0"),
    dbc.Row([
        dbc.Col([
            dbc.Row(html.H3('NOVEDADES DE LOCAL', style={'textAlign': 'center', 'background-color': '#267300'})),
            dcc.Loading(id="ls-loading-3", children=[generar_tabla(df, 'datatable1')]),
            dbc.Row(html.H3('NOVEDADES GENERALES', style={'textAlign': 'center', 'background-color': '#267300'})),
            dcc.Loading(id="ls-loading-1", children=[generar_tabla(df_generales, 'datatable2')])
        ], md=5),
        dbc.Col([
            dcc.Loading(id="ls-loading-4", children=[dcc.Graph(id='mapa', figure=crear_mapa(df, 3))])
        ], md=4),
        dbc.Col([
            dcc.Loading(id="ls-loading-5", children=[dcc.Graph(id='pie_3', figure=graf_nov_loc(df))]),
            dcc.Loading(id="ls-loading-6", children=[dcc.Graph(id='pie4', figure=graf_nov_loc(df_generales))])
        ], md=3)
    ], className="g-0"),
    dcc.Interval(id='interval-component', interval=60 * 1000, n_intervals=0)
])

@app2.callback(
    [Output('datatable1', 'data'), Output('datatable2', 'data'), Output('mapa', 'figure'),
     Output('pie_3', 'figure'), Output('pie4', 'figure')],
    [Input('interval-component', 'n_intervals'), Input('selec_distrito', 'value'),
     Input('Subsanada', 'value'), Input('datatable1', 'active_cell'),
     Input('datatable1', 'derived_virtual_data')],
    [State('datatable1', 'data')]
)
def actualizar_tabla(n_intervals, distrito, subsanada, active_cell, nueva_tabla, tabla):
    try:
        df = get_data(distrito, subsanada)
        df_generales = get_data_generales(distrito, subsanada)
        df1 = df.drop(columns=['geom'], errors='ignore')
        df1_generales = df_generales.drop(columns=['geom'], errors='ignore')
        
        zoom = 3
        lat_cen, lon_cen = None, None
        if active_cell and nueva_tabla:
            tabla = nueva_tabla
            lat_cen = tabla[active_cell['row']]['lat']
            lon_cen = tabla[active_cell['row']]['lon']
            zoom = 18
        
        return (
            df1.to_dict('records'),
            df1_generales.to_dict('records'),
            crear_mapa(df, zoom, lat_cen, lon_cen),
            graf_nov_loc(df),
            graf_nov_loc(df_generales)
        )
    except Exception as e:
        print(f"Error en callback: {e}")
        return [], [], {}, {}, {}

@app2.callback(
    [Output('alerta', 'is_open'), Output('audio', 'autoPlay'), Output('row-count', 'data')],
    [Input('interval-component', 'n_intervals')],
    [State('row-count',  'data')]
)
def mostrar_alerta(n_intervals, row_count):
    try:
        df = get_data()
        new_count = len(df)
        count_data = row_count.get('count', 0)
        if count_data != new_count:
            return True, True, {'count': new_count}
        return False, False, {'count': new_count}
    except Exception as e:
        print(f"Error en alerta: {e}")
        return False, False, row_count

app2.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [27/Jun/2025 10:22:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 10:22:25] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_9_2m1739804920.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 10:22:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 10:22:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 10:22:26] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2025 10:22:26] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Jun/2025 10:22:26] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Jun/2025 10:22:26] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
[2025-06-27 10:22:26,391] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/esteban/.local/share/v